In [ ]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,918 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 2,307 kB in 4s (654 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session (if not already done)
spark = SparkSession.builder \
    .appName("TerraTrend-HousePricePrediction") \
    .getOrCreate()

# Read the CSV file
df = spark.read.csv(r"/content/drive/MyDrive/TerraTrend-HousePricePrediction/house_prices.csv", header=True, inferSchema=True)

# Show the schema and first few rows
df.printSchema()
df.show(5)

root
 |-- Index: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Amount(in rupees): string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Floor: string (nullable = true)
 |-- Transaction: string (nullable = true)
 |-- Furnishing: string (nullable = true)
 |-- facing: string (nullable = true)
 |-- overlooking: string (nullable = true)
 |-- Society: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Car Parking: string (nullable = true)
 |-- Ownership: string (nullable = true)
 |-- Super Area: string (nullable = true)
 |-- Dimensions: string (nullable = true)
 |-- Plot Area: string (nullable = true)

+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+-------

In [ ]:
from pyspark.sql import functions as F

# Count total number of duplicate rows in the DataFrame
duplicate_rows_count = df.groupBy(df.columns) \
                        .count() \
                        .filter(F.col("count") > 1) \
                        .withColumn("dup_count", F.col("count") - 1) \
                        .agg(F.sum("dup_count")) \
                        .collect()[0][0]

# If there are no duplicates, duplicate_rows_count will be None
duplicate_rows_count = duplicate_rows_count if duplicate_rows_count is not None else 0

print(duplicate_rows_count)


0


In [ ]:


# Compute missing values per column
missing_values = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])

# Show as a PySpark DataFrame (like pandas Series output)
missing_values.show()

# If you'd like the result as a Python dictionary
missing_counts = missing_values.collect()[0].asDict()
print(missing_counts)


+-----+-----+-----------+-----------------+-----------------+--------+-----------+------+-----+-----------+----------+------+-----------+-------+--------+-------+-----------+---------+----------+----------+---------+
|Index|Title|Description|Amount(in rupees)|Price (in rupees)|location|Carpet Area|Status|Floor|Transaction|Furnishing|facing|overlooking|Society|Bathroom|Balcony|Car Parking|Ownership|Super Area|Dimensions|Plot Area|
+-----+-----+-----------+-----------------+-----------------+--------+-----------+------+-----+-----------+----------+------+-----------+-------+--------+-------+-----------+---------+----------+----------+---------+
|    0|    0|       3023|                0|            17664|       0|      80670|   615| 7079|         83|      2899| 70233|      81436| 109677|     828|  48936|     103357|    65517|    107685|    187528|   187528|
+-----+-----+-----------+-----------------+-----------------+--------+-----------+------+-----+-----------+----------+------+-------

In [ ]:
column_names = [
    'Title',
    'Description',
    'Amount(in rupees)',
    'Price (in rupees)',
    'location',
    'Carpet Area',
    'Status',
    'Floor',
    'Transaction',
    'Furnishing',
    'facing',
    'overlooking',
    'Society',
    'Bathroom',
    'Balcony',
    'Car Parking',
    'Ownership',
    'Super Area',
    'Dimensions',
    'Plot Area'
]

print(column_names)


['Title', 'Description', 'Amount(in rupees)', 'Price (in rupees)', 'location', 'Carpet Area', 'Status', 'Floor', 'Transaction', 'Furnishing', 'facing', 'overlooking', 'Society', 'Bathroom', 'Balcony', 'Car Parking', 'Ownership', 'Super Area', 'Dimensions', 'Plot Area']


In [ ]:
total_rows = df.count()


In [ ]:
from pyspark.sql import functions as F

for col in column_names:
    if col in df.columns:
        null_count = df.filter(F.col(col).isNull()).count()
        percentage_null = (null_count / total_rows) * 100 if total_rows > 0 else 0
        print(f"Column '{col}': {null_count} nulls ({percentage_null:.2f}%)")
    else:
        print(f"Warning: Column '{col}' not found in DataFrame.")


Column 'Title': 0 nulls (0.00%)
Column 'Description': 3023 nulls (1.61%)
Column 'Amount(in rupees)': 0 nulls (0.00%)
Column 'Price (in rupees)': 17664 nulls (9.42%)
Column 'location': 0 nulls (0.00%)
Column 'Carpet Area': 80670 nulls (43.02%)
Column 'Status': 615 nulls (0.33%)
Column 'Floor': 7079 nulls (3.77%)
Column 'Transaction': 83 nulls (0.04%)
Column 'Furnishing': 2899 nulls (1.55%)
Column 'facing': 70233 nulls (37.45%)
Column 'overlooking': 81436 nulls (43.43%)
Column 'Society': 109677 nulls (58.48%)
Column 'Bathroom': 828 nulls (0.44%)
Column 'Balcony': 48936 nulls (26.09%)
Column 'Car Parking': 103357 nulls (55.11%)
Column 'Ownership': 65517 nulls (34.94%)
Column 'Super Area': 107685 nulls (57.42%)
Column 'Dimensions': 187528 nulls (100.00%)
Column 'Plot Area': 187528 nulls (100.00%)


In [ ]:
df = df.drop(*["Dimensions", "Plot Area","Description"])


In [ ]:
df.groupBy('Amount(in rupees)').count().orderBy(F.desc('count')).show()

+-----------------+-----+
|Amount(in rupees)|count|
+-----------------+-----+
|   Call for Price| 9683|
|          85 Lac | 5264|
|          65 Lac | 4229|
|          60 Lac | 3869|
|          70 Lac | 3801|
|          35 Lac | 3369|
|          75 Lac | 3144|
|          90 Lac | 3143|
|          40 Lac | 3098|
|          50 Lac | 3006|
|         1.75 Cr | 2879|
|          30 Lac | 2790|
|          25 Lac | 2771|
|          51 Lac | 2682|
|          45 Lac | 2665|
|         1.30 Cr | 2561|
|          22 Lac | 2537|
|         2.35 Cr | 2361|
|          23 Lac | 2312|
|          88 Lac | 2310|
+-----------------+-----+
only showing top 20 rows



In [ ]:
df = df.filter(df['Amount(in rupees)'] != 'Call for Price')


In [ ]:
num_rows = df.count()
num_cols = len(df.columns)
print((num_rows, num_cols))


(177848, 18)


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

# This function will be used as a UDF
def treat_price(parts):
    try:
        if parts is None or len(parts) == 0:
            return None
        if len(parts) == 1:
            return round(float(parts[0]), 2)
        val, unit = parts[0], parts[1]
        if unit == 'Cr':
            return round(float(val) * 100, 2)
        elif unit == 'Lac':
            return round(float(val), 2)
        else:
            return None
    except:
        return None

# Register as a UDF
treat_price_udf = F.udf(treat_price, DoubleType())

# Split column and apply UDF to create "Final Amount"
df = df.withColumn(
    "Final Amount",
    treat_price_udf(F.split(F.col("Amount(in rupees)"), " "))
)


In [ ]:
df = df.drop("Amount(in rupees)")


In [ ]:
location_cnt = df.groupBy("location").count().orderBy(F.desc("count"))
location_cnt.show()


+-------------+-----+
|     location|count|
+-------------+-----+
|    new-delhi|24944|
|    bangalore|23262|
|      kolkata|21605|
|      gurgaon|18845|
|    ahmedabad|12614|
|    hyderabad|11147|
|      chennai|10163|
|       jaipur| 7867|
|greater-noida| 4490|
|    faridabad| 3733|
|     vadodara| 2361|
|        surat| 2180|
|         pune| 2177|
|        thane| 1869|
|       mumbai| 1814|
|visakhapatnam| 1729|
|       mohali| 1479|
|     zirakpur| 1478|
|   chandigarh| 1402|
|        noida| 1247|
+-------------+-----+
only showing top 20 rows



In [ ]:
location_top25 = location_cnt.limit(25)
location_top25.show()


+-------------+-----+
|     location|count|
+-------------+-----+
|    new-delhi|24944|
|    bangalore|23262|
|      kolkata|21605|
|      gurgaon|18845|
|    ahmedabad|12614|
|    hyderabad|11147|
|      chennai|10163|
|       jaipur| 7867|
|greater-noida| 4490|
|    faridabad| 3733|
|     vadodara| 2361|
|        surat| 2180|
|         pune| 2177|
|        thane| 1869|
|       mumbai| 1814|
|visakhapatnam| 1729|
|       mohali| 1479|
|     zirakpur| 1478|
|   chandigarh| 1402|
|        noida| 1247|
+-------------+-----+
only showing top 20 rows



In [ ]:
# Collect the top 25 location names into a Python list
top25_locations = [row['location'] for row in location_top25.collect()]

# Filter df to only those locations
df = df.filter(df['location'].isin(top25_locations))


In [ ]:
# Get distinct values from the "Transaction" column
unique_transactions = df.select("Transaction").distinct()
unique_transactions.show()


+------------+
| Transaction|
+------------+
|  Rent/Lease|
|       Other|
|      Resale|
|New Property|
|        NULL|
+------------+



In [ ]:
# Filter out rows where 'Transaction' is 'Rent/Lease' or 'Other'
df = df.filter((df['Transaction'] != 'Rent/Lease') & (df['Transaction'] != 'Other'))

# Get the shape (number of rows and columns)
num_rows = df.count()
num_cols = len(df.columns)
print((num_rows, num_cols))


(160877, 18)


In [ ]:
# Calculate NULLs per column
missing_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])

# Show as a table (like pandas Series)
missing_counts.show()

# Or, to get it as a Python dictionary (similar to pandas output)
missing_dict = missing_counts.collect()[0].asDict()
print(missing_dict)

+-----+-----+-----------------+--------+-----------+------+-----+-----------+----------+------+-----------+-------+--------+-------+-----------+---------+----------+------------+
|Index|Title|Price (in rupees)|location|Carpet Area|Status|Floor|Transaction|Furnishing|facing|overlooking|Society|Bathroom|Balcony|Car Parking|Ownership|Super Area|Final Amount|
+-----+-----+-----------------+--------+-----------+------+-----+-----------+----------+------+-----------+-------+--------+-------+-----------+---------+----------+------------+
|    0|    0|             6217|       0|      68667|   518| 6190|          0|      1695| 56896|      66265|  92014|     579|  40921|      83807|    53584|     92260|           0|
+-----+-----+-----------------+--------+-----------+------+-----+-----------+----------+------+-----------+-------+--------+-------+-----------+---------+----------+------------+

{'Index': 0, 'Title': 0, 'Price (in rupees)': 6217, 'location': 0, 'Carpet Area': 68667, 'Status': 518, 

In [ ]:
df = df.drop("Index")


In [ ]:
# Define your column_names list if it isn't already defined
column_names = [
    'Title', 'Description', 'Amount(in rupees)', 'Price (in rupees)',
    'location', 'Carpet Area', 'Status', 'Floor', 'Transaction',
    'Furnishing', 'facing', 'overlooking', 'Society', 'Bathroom',
    'Balcony', 'Car Parking', 'Ownership', 'Super Area', 'Dimensions',
    'Plot Area'
]

# Make sure total_rows is defined
total_rows = df.count()

for col in column_names:
    if col in df.columns:
        null_count = df.filter(F.col(col).isNull()).count()
        percentage_null = (null_count / total_rows) * 100 if total_rows > 0 else 0
        print(f"Column '{col}': {null_count} nulls ({percentage_null:.2f}%)")
    else:
        print(f"Warning: Column '{col}' not found in DataFrame.")

Column 'Title': 0 nulls (0.00%)
Column 'Price (in rupees)': 6217 nulls (3.86%)
Column 'location': 0 nulls (0.00%)
Column 'Carpet Area': 68667 nulls (42.68%)
Column 'Status': 518 nulls (0.32%)
Column 'Floor': 6190 nulls (3.85%)
Column 'Transaction': 0 nulls (0.00%)
Column 'Furnishing': 1695 nulls (1.05%)
Column 'facing': 56896 nulls (35.37%)
Column 'overlooking': 66265 nulls (41.19%)
Column 'Society': 92014 nulls (57.20%)
Column 'Bathroom': 579 nulls (0.36%)
Column 'Balcony': 40921 nulls (25.44%)
Column 'Car Parking': 83807 nulls (52.09%)
Column 'Ownership': 53584 nulls (33.31%)
Column 'Super Area': 92260 nulls (57.35%)


In [ ]:
# Count number of nulls in the "Final Amount" column
null_count = df.filter(F.col("Final Amount").isNull()).count()
print(null_count)

0


In [ ]:
# Count the number of nulls in the "Price (in rupees)" column
null_count = df.filter(F.col("Price (in rupees)").isNull()).count()
print(null_count)

6217


In [ ]:
null_count = df.filter(F.col("facing").isNull()).count()
print(null_count)

56896


In [ ]:
df = df.withColumn(
    "facing",
    F.when(F.col("facing").isNull(), F.lit("NA")).otherwise(F.col("facing"))
)

In [ ]:
facing_counts = df.groupBy('facing').count().orderBy(F.desc('count'))
facing_counts.show()

+------------+-----+
|      facing|count|
+------------+-----+
|          NA|56896|
|        East|48402|
|North - East|21949|
|       North|13987|
|        West| 7734|
|       South| 4023|
|North - West| 3574|
|South - East| 2377|
| South -West| 1935|
+------------+-----+



In [ ]:
df = df.drop("overlooking")


In [ ]:
num_rows = df.count()
num_cols = len(df.columns)
print((num_rows, num_cols))


(160877, 16)


In [ ]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Floor: string (nullable = true)
 |-- Transaction: string (nullable = true)
 |-- Furnishing: string (nullable = true)
 |-- facing: string (nullable = true)
 |-- Society: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Car Parking: string (nullable = true)
 |-- Ownership: string (nullable = true)
 |-- Super Area: string (nullable = true)
 |-- Final Amount: double (nullable = true)



In [ ]:
df = df.drop("Car Parking")


In [ ]:
# Get distinct values in the "Transaction" column
unique_transactions = df.select("Transaction").distinct()

# Show them
unique_transactions.show()

+------------+
| Transaction|
+------------+
|      Resale|
|New Property|
+------------+



In [ ]:
# from pyspark.sql import functions as F

# df = df.withColumn(
#     "Transaction",
#     F.when(F.col("Transaction") == "0", 0)
#      .when(F.col("Transaction") == "Other", 1)
#      .when(F.col("Transaction") == "Rent/Lease", 2)
#      .when(F.col("Transaction") == "Resale", 3)
#      .when(F.col("Transaction") == "New Property", 4)
#      .otherwise(F.col("Transaction"))  # keep original if not listed
# )
df = df.withColumn(
    "Transaction",
    F.when(F.col("Transaction") == "0", 0)
     .when(F.col("Transaction") == "Other", 1)
     .when(F.col("Transaction") == "Rent/Lease", 2)
     .when(F.col("Transaction") == "Resale", 3)
     .when(F.col("Transaction") == "New Property", 4)
     .otherwise(-1)   # Mark all unknowns as -1 (or another integer)
     .cast("int")
)



In [ ]:
# Get distinct values in the "Transaction" column
unique_transactions = df.select("Transaction").distinct()

# Show them
unique_transactions.show()

+-----------+
|Transaction|
+-----------+
|          3|
|          4|
+-----------+



In [ ]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Floor: string (nullable = true)
 |-- Transaction: integer (nullable = false)
 |-- Furnishing: string (nullable = true)
 |-- facing: string (nullable = true)
 |-- Society: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Ownership: string (nullable = true)
 |-- Super Area: string (nullable = true)
 |-- Final Amount: double (nullable = true)



In [ ]:
# Get distinct values in the "Furnishing" column
unique_furnishing = df.select("Furnishing").distinct()
unique_furnishing.show()

+--------------+
|    Furnishing|
+--------------+
|Semi-Furnished|
|     Furnished|
|   Unfurnished|
|          NULL|
+--------------+



In [ ]:
# from pyspark.sql import functions as F

# df = df.withColumn(
#     "Furnishing",
#     F.when(F.col("Furnishing") == "0", 0)
#      .when(F.col("Furnishing") == "Unfurnished", 1)
#      .when(F.col("Furnishing") == "Semi-Furnished", 2)
#      .when(F.col("Furnishing") == "Furnished", 3)
#      .otherwise(F.col("Furnishing"))
# )
from pyspark.sql import functions as F

df = df.withColumn(
    "Furnishing",
    F.when(F.col("Furnishing") == "0", 0)
     .when(F.col("Furnishing") == "Unfurnished", 1)
     .when(F.col("Furnishing") == "Semi-Furnished", 2)
     .when(F.col("Furnishing") == "Furnished", 3)
     .otherwise(-1)        # <- all others, including NULL, set to -1
     .cast("int")          # <- ensures whole column is int
)



In [ ]:
# Get distinct values in the "Status" column
unique_status = df.select("Status").distinct()
unique_status.show()

+-------------+
|       Status|
+-------------+
|Ready to Move|
|         NULL|
+-------------+



In [ ]:
df = df.withColumn(
    "Status",
    F.when(F.col("Status").isNull(), F.lit("0")).otherwise(F.col("Status"))
)

In [ ]:
unique_status = df.select("Status").distinct()
unique_status.show()

+-------------+
|       Status|
+-------------+
|Ready to Move|
|            0|
+-------------+



In [ ]:
# df = df.withColumn(
#     "Status",
#     F.when(F.col("Status") == "0", 0)
#      .when(F.col("Status") == "Ready to Move", 1)
#      .otherwise(F.col("Status"))
# )
from pyspark.sql import functions as F

df = df.withColumn(
    "Status",
    F.when(F.col("Status") == "0", 0)
     .when(F.col("Status") == "Ready to Move", 1)
     .otherwise(-1)           # assign -1 (or another int) for all other/unknown/NULLs
     .cast("int")             # ensure the entire column is int type
)


In [ ]:
df = df.drop("Floor")


In [ ]:
# Get distinct values in the "facing" column
unique_facing = df.select("facing").distinct()
unique_facing.show()

+------------+
|      facing|
+------------+
|South - East|
|          NA|
| South -West|
|       South|
|        East|
|North - West|
|        West|
|North - East|
|       North|
+------------+



In [ ]:
from pyspark.sql import functions as F

# Your mapping as a dictionary
lable_map = {
    'East': 3,
    'West': 2,
    'North': 2,
    'South': 4,
    'North - East': 5,
    'North - West': 5,
    'South - East': 0,
    'South -West': 1,
    'NA': 0
}

# Create PySpark map expression
mapping_expr = F.create_map([F.lit(x) for pair in lable_map.items() for x in pair])

# Overwrite "facing" with mapped integer values (or keep as is if unmapped)
df = df.withColumn(
    "facing",
    mapping_expr.getItem(F.col("facing")).cast("int")
)


/usr/local/lib/python3.11/dist-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [ ]:
unique_ownership = df.select("Ownership").distinct()
unique_ownership.show()

+--------------------+
|           Ownership|
+--------------------+
|            Freehold|
|   Power Of Attorney|
|Co-operative Society|
|           Leasehold|
|                NULL|
+--------------------+



In [ ]:
# df = df.withColumn(
#     "Ownership",
#     F.when(F.col("Ownership") == "Freehold", 0)
#      .when(F.col("Ownership") == "Leasehold", 1)
#      .when(F.col("Ownership") == "Power Of Attorney", 2)
#      .when(F.col("Ownership") == "Co-operative Society", 3)
#      .otherwise(F.col("Ownership"))
# )
from pyspark.sql import functions as F

df = df.withColumn(
    "Ownership",
    F.when(F.col("Ownership") == "Freehold", 0)
     .when(F.col("Ownership") == "Leasehold", 1)
     .when(F.col("Ownership") == "Power Of Attorney", 2)
     .when(F.col("Ownership") == "Co-operative Society", 3)
     .otherwise(-1)    # Unmapped/NULLs get -1 (or whatever you like)
     .cast("int")      # Force the column to integer type
)



In [ ]:
unique_ownership = df.select("Ownership").distinct()
unique_ownership.show()

+---------+
|Ownership|
+---------+
|       -1|
|        1|
|        3|
|        2|
|        0|
+---------+



In [ ]:
num_rows = df.count()
num_cols = len(df.columns)
print((num_rows, num_cols))


(160877, 14)


In [ ]:
# Get distinct values in the "Bathroom" column
unique_bathroom = df.select("Bathroom").distinct()
unique_bathroom.show()


+--------+
|Bathroom|
+--------+
|       7|
|       3|
|    > 10|
|       8|
|       5|
|       6|
|       9|
|       1|
|      10|
|       4|
|       2|
|    NULL|
+--------+



In [ ]:
from pyspark.sql import functions as F

df = df.withColumn(
    "Bathroom",
    F.when(F.col("Bathroom") == "> 10", 11).otherwise(F.col("Bathroom"))
)


In [ ]:
# Get distinct values in the "Bathroom" column
unique_bathroom = df.select("Bathroom").distinct()
unique_bathroom.show()


+--------+
|Bathroom|
+--------+
|       7|
|      11|
|       3|
|       8|
|       5|
|       6|
|       9|
|       1|
|      10|
|       4|
|       2|
|    NULL|
+--------+



In [ ]:
null_count = df.filter(F.col("Bathroom").isNull()).count()
print(null_count)

579


In [ ]:
unique_balcony = df.select("Balcony").distinct()
unique_balcony.show()

+-------+
|Balcony|
+-------+
|      7|
|      3|
|   > 10|
|      5|
|      6|
|      1|
|     10|
|      4|
|      2|
|      8|
|      9|
|   NULL|
+-------+



In [ ]:
df = df.withColumn(
    "Balcony",
    F.when(F.col("Balcony") == "> 10", 11).otherwise(F.col("Balcony"))
)

In [ ]:
unique_balcony = df.select("Balcony").distinct()
unique_balcony.show()

+-------+
|Balcony|
+-------+
|      7|
|     11|
|      3|
|      5|
|      6|
|      1|
|     10|
|      4|
|      2|
|      8|
|      9|
|   NULL|
+-------+



In [ ]:
# Get a list of (column, null_count) tuples
null_counts = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df.columns
])

null_counts.show()

+-----+-----------------+--------+-----------+------+-----------+----------+------+-------+--------+-------+---------+----------+------------+
|Title|Price (in rupees)|location|Carpet Area|Status|Transaction|Furnishing|facing|Society|Bathroom|Balcony|Ownership|Super Area|Final Amount|
+-----+-----------------+--------+-----------+------+-----------+----------+------+-------+--------+-------+---------+----------+------------+
|    0|             6217|       0|      68667|     0|          0|         0|     0|  92014|     579|  40921|        0|     92260|           0|
+-----+-----------------+--------+-----------+------+-----------+----------+------+-------+--------+-------+---------+----------+------------+



In [ ]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: integer (nullable = false)
 |-- Transaction: integer (nullable = false)
 |-- Furnishing: integer (nullable = false)
 |-- facing: integer (nullable = true)
 |-- Society: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Ownership: integer (nullable = false)
 |-- Super Area: string (nullable = true)
 |-- Final Amount: double (nullable = true)



In [ ]:
df=df.drop('Society')

In [ ]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: integer (nullable = false)
 |-- Transaction: integer (nullable = false)
 |-- Furnishing: integer (nullable = false)
 |-- facing: integer (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Ownership: integer (nullable = false)
 |-- Super Area: string (nullable = true)
 |-- Final Amount: double (nullable = true)



In [ ]:
df.show()

+--------------------+-----------------+--------+-----------+------+-----------+----------+------+--------+-------+---------+----------+------------+
|               Title|Price (in rupees)|location|Carpet Area|Status|Transaction|Furnishing|facing|Bathroom|Balcony|Ownership|Super Area|Final Amount|
+--------------------+-----------------+--------+-----------+------+-----------+----------+------+--------+-------+---------+----------+------------+
|1 BHK Ready to Oc...|             6000|   thane|   500 sqft|     1|          3|         1|     0|       1|      2|       -1|      NULL|        42.0|
|2 BHK Ready to Oc...|            13799|   thane|   473 sqft|     1|          3|         2|     3|       2|   NULL|        0|      NULL|        98.0|
|2 BHK Ready to Oc...|            17500|   thane|   779 sqft|     1|          3|         1|     3|       2|   NULL|        0|      NULL|       140.0|
|1 BHK Ready to Oc...|             NULL|   thane|   530 sqft|     1|          3|         1|     0|  

In [ ]:
df.coalesce(1).write.csv("/content/drive/MyDrive/TeeraTrend-HousePricePrediction_Spark_Cleaning_001/TerraTrend-HousePricePrediction_Spark001.csv", header=True, mode="overwrite")


In [ ]:
pandas_df = df.toPandas()

In [ ]:
pandas_df.shape

(160877, 13)

In [ ]:
pandas_df.head()

,Title,Price (in rupees),location,Carpet Area,Status,Transaction,Furnishing,facing,Bathroom,Balcony,Ownership,Super Area,Final Amount
0,1 BHK Ready to Occupy Flat for sale in Srushti...,6000,thane,500 sqft,1,3,1,0,1,2,-1,None,42.0
1,2 BHK Ready to Occupy Flat for sale in Dosti V...,13799,thane,473 sqft,1,3,2,3,2,None,0,None,98.0
2,2 BHK Ready to Occupy Flat for sale in Sunrise...,17500,thane,779 sqft,1,3,1,3,2,None,0,None,140.0
3,1 BHK Ready to Occupy Flat for sale Kasheli,None,thane,530 sqft,1,3,1,0,1,1,-1,None,25.0
4,2 BHK Ready to Occupy Flat for sale in TenX Ha...,18824,thane,635 sqft,1,3,1,2,2,None,3,None,160.0
